In [62]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

url = "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv"
response = requests.get(url)
data = StringIO(response.text)
df = pd.read_csv(data)
df

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE
...,...,...,...,...,...,...,...
339,Gentoo,Biscoe,NaN,NaN,NaN,NaN,NaN
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,FEMALE
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,MALE
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,FEMALE


In [64]:
df.isnull().sum()

species               0
island                0
bill_length_mm        2
bill_depth_mm         2
flipper_length_mm     2
body_mass_g           2
sex                  11
dtype: int64

In [66]:
label_encoders={}
for column in df.columns:
  if df[column].dtype == 'object':
    label_encoders[column] = LabelEncoder()
    df[column] = label_encoders[column].fit_transform(df[column])
df

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,0,2,39.1,18.7,181.0,3750.0,1
1,0,2,39.5,17.4,186.0,3800.0,0
2,0,2,40.3,18.0,195.0,3250.0,0
3,0,2,NaN,NaN,NaN,NaN,2
4,0,2,36.7,19.3,193.0,3450.0,0
...,...,...,...,...,...,...,...
339,2,0,NaN,NaN,NaN,NaN,2
340,2,0,46.8,14.3,215.0,4850.0,0
341,2,0,50.4,15.7,222.0,5750.0,1
342,2,0,45.2,14.8,212.0,5200.0,0


In [76]:
df['sex'] = df['sex'].fillna(df['sex'].median()) #평균값
df['bill_length_mm'] = df['bill_length_mm'].fillna(df['bill_length_mm'].median()) #평균값
df['bill_depth_mm'] = df['bill_depth_mm'].fillna(df['bill_depth_mm'].median()) #평균값
df['flipper_length_mm'] = df['flipper_length_mm'].fillna(df['flipper_length_mm'].median()) #평균값
df['body_mass_g'] = df['body_mass_g'].fillna(df['body_mass_g'].median()) #평균값
df

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,0,2,39.10,18.7,181.0,3750.0,1
1,0,2,39.50,17.4,186.0,3800.0,0
2,0,2,40.30,18.0,195.0,3250.0,0
3,0,2,44.45,17.3,197.0,4050.0,2
4,0,2,36.70,19.3,193.0,3450.0,0
...,...,...,...,...,...,...,...
339,2,0,44.45,17.3,197.0,4050.0,2
340,2,0,46.80,14.3,215.0,4850.0,0
341,2,0,50.40,15.7,222.0,5750.0,1
342,2,0,45.20,14.8,212.0,5200.0,0


In [96]:
cnt = 343;

for cnt in 343:
  if df[column].dtype == 'object':
    label_encoders[column] = LabelEncoder()
    df[column] = label_encoders[column].fit_transform(df[column])

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [41]:
X = df.drop(columns=['island', 'species'])
y = df['sex']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

In [43]:
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_test = torch.tensor(y_test.values, dtype=torch.long)

In [51]:
X.shape, y.shape

((344, 5), (344,))

In [53]:
#모델링
class BP_Classifier(nn.Module):
    def __init__(self):
        super(BP_Classifier, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 2)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = BP_Classifier()

In [55]:
#손실값으로 optimizer가중치 조절 Adam 사용
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [80]:
#실 학습 시작
epochs = 30
train_loss_list = []
test_acc_list = []

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = loss_fn(output, y_train)
    loss.backward()
    optimizer.step()
    
    train_loss_list.append(loss.item())
    
    model.eval()
    with torch.no_grad():
        preds = model(X_test)
        _, preds_class = torch.max(preds, 1)
        acc = (preds_class == y_test).float().mean()
        test_acc_list.append(acc.item())
    
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}, Test Acc: {acc.item()*100:.2f}%")

IndexError: Target 2 is out of bounds.